## Preparación del entorno

In [23]:
%matplotlib inline
import numpy as np

import sklearn

import pandas as pd

Si el entorno está correctamente instalado, las líneas de código anteriores deben importar los paquetes sin ningún error.

Nota: para el resto de las preguntas y soluciones de código, puede ingresar más celdas si lo considera necesario.


## Carga y estudio de datos

Cargue los datos desde el archivo *adult_data.csv*. Para esto puede utilizar la librería *pandas* con su función *read_csv*.

In [24]:
df = pd.read_csv('adult_data.csv',skipinitialspace=True)

Imprima los nombres de las columnas (atributos), e investigue la documentación para entender que significa cada uno de ellos.

In [25]:
df.columns

Index([u'age', u'workclass', u'fnlwgt', u'education', u'education-num',
       u'marital-status', u'occupation', u'relationship', u'race', u'sex',
       u'capital-gain', u'capital-loss', u'hours-per-week', u'native-country',
       u'income'],
      dtype='object')

**PREGUNTA: A continuación realice algunas conjeturas de cuáles pueden llegar a ser los atributos de mayor utilidad para predecir el nivel de ingresos (income) de una persona.**

**RESPUESTA:**

capital-gain
capital-loss
education
occupation

## Extracción de atributos

Separar la columna **income** en un array **y** que será utilizada como atributo clase:

In [26]:
y = np.array(df.income)

Eliminar la columna **fnlwgt** ya que no aporta a la solución del problema. También eliminar la columna **education-num** ya que duplica la información de la columna 'education'. Por último, eliminar la columna **income** ya que es la columna que contiene la clase que se pretende predecir:

In [27]:
del df['fnlwgt']
del df['education-num']
del df['income']

Los atributos cuyos valores son categorías ('workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'), deben de transformarse a valores numéricos para poder ser utilizados como entradas en los modelos de scikit-learn.

**PREGUNTA: Por qué no es apropiado transformar un atributo de categoría en simples índices numéricos?**

**RESPUESTA: Porque se generaría una relación de órden y magnitud que el algoritmo de aprendizaje puede tomar y no es real...**

Utilice las clases *LabelEncoder* y *OneHotEncoder* del paquete *preprocessing* de *sklearn* para transformar los atributos de categorías en atributos numéricos. Guarde los datos de entrada en una matriz **X**.

In [28]:
import sklearn.preprocessing

aTransformar = ['workclass', 'education', 'marital-status', 'sex','occupation', 'native-country', 'relationship', 'race']

le = {}

for t in aTransformar:
    le[t] = sklearn.preprocessing.LabelEncoder()
    le[t].fit(df[t])
    df[t] = le[t].transform(df[t])
    
features = [df.columns.get_loc(f) for f in aTransformar]

ohe = sklearn.preprocessing.OneHotEncoder(categorical_features=features, sparse=False)
X = ohe.fit_transform(df)

In [29]:
X

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.17400000e+03,   0.00000000e+00,   4.00000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.30000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   4.00000000e+01],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   4.00000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   6.00000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   5.50000000e+01]])

**PREGUNTA: Cuántos y cuáles son los nuevos atributos del dataset?**

**RESPUESTA:**

In [30]:
len(X[0])

103

In [31]:
def obtenerNombreFeature(i):
    
    if i >= ohe.feature_indices_[-1]:
        
        noCategoricalIndex = i - ohe.feature_indices_[-1]
        
        j = -1
        n = -1
        while j != noCategoricalIndex:
            n += 1
            if df.columns[n] not in aTransformar:
                j += 1
        return df.columns[n]

    else:
        enumerated_features = reversed([x for x in enumerate(ohe.feature_indices_)])
        pos, categoria = next((pos, j) for (pos, j) in enumerated_features if j <= i)
        nombreCategoria = df.columns[sorted(ohe.categorical_features)[pos]]
        
        etiqueta = i - categoria
        nombreEtiqueta = le[nombreCategoria].classes_[etiqueta]
        
        return nombreCategoria + "/" + nombreEtiqueta

print([obtenerNombreFeature(i) for i in range(len(X[0]))])
        

['workclass/?', 'workclass/Federal-gov', 'workclass/Local-gov', 'workclass/Private', 'workclass/Self-emp-inc', 'workclass/Self-emp-not-inc', 'workclass/State-gov', 'workclass/Without-pay', 'education/10th', 'education/11th', 'education/12th', 'education/1st-4th', 'education/5th-6th', 'education/7th-8th', 'education/9th', 'education/Assoc-acdm', 'education/Assoc-voc', 'education/Bachelors', 'education/Doctorate', 'education/HS-grad', 'education/Masters', 'education/Preschool', 'education/Prof-school', 'education/Some-college', 'marital-status/Divorced', 'marital-status/Married-AF-spouse', 'marital-status/Married-civ-spouse', 'marital-status/Married-spouse-absent', 'marital-status/Never-married', 'marital-status/Separated', 'marital-status/Widowed', 'occupation/?', 'occupation/Adm-clerical', 'occupation/Armed-Forces', 'occupation/Craft-repair', 'occupation/Exec-managerial', 'occupation/Farming-fishing', 'occupation/Handlers-cleaners', 'occupation/Machine-op-inspct', 'occupation/Other-ser

## Partición de datos

Para poder entrenar y testear un algoritmo de aprendizaje, es necesario primero particionar los datos en dos conjuntos disjuntos de entrenamiento y testeo. Separe aleatoriamente un 25% de los datos para testeo, llame a los atributos de entrada como **X_test** y al vector de salida esperado **y_test**. El 75% restante se utilizará para el entrenamiento, nombre a la matriz con los datos de entrada como **X_train** y al vector de salida correspondiente como **y_train**.
Para esto puede utilizar la función *train_test_split* del paquete *cross_validation* de *sklearn*:

In [32]:
import sklearn.cross_validation
X_train,X_test,y_train,y_test = sklearn.cross_validation.train_test_split(X,y, test_size=0.25)

C:\Users\bgarate.SOFT\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Examine el tamaño de las matrices y vectores generados:

In [33]:
len(X_train)

3750

In [ ]:
len(y_train)

In [ ]:
len(X_test)

In [ ]:
len(y_test)

## Entrenamiento

Ahora que tenemos particionados los datos en entrenamiento y testeo, podemos comenzar a entrenar los algoritmos.

Genere un modelo 'dt' entrenando un algoritmo de árboles de decisión (ver el paquete *tree* de *sklearn*) con el vector de entrada X_train y el vector de salida y_train. Utilice los valores por defecto:

In [ ]:
import sklearn.tree as tree

dt = tree.DecisionTreeClassifier()
dt.fit(X_train,y_train)

Genere un modelo 'nb' entrenando un algoritmo de Naive Bayes (ver el paquete *naive_bayes* de *sklearn*) con el vector de entrada X_train y el vector de salida y_train. Utilice los valores por defecto:

In [ ]:
import sklearn.naive_bayes as naive_bayes


nb = naive_bayes.GaussianNB()
nb.fit(X_train, y_train)

Genere un modelo 'svc' entrenando un algoritmo de Support Vector Machines (ver el paquete *svm* de *sklearn*) con el vector de entrada X_train y el vector de salida y_train. Utilice los valores por defecto:

In [ ]:
import sklearn.svm as svm

svc = svm.SVC()
svc.fit(X_train, y_train)

## Testing

Luego de tener los modelos entrenados, podemos medir qué tan bien funcionan los modelos (su capacidad de predicción) utlizando medidas standard como accuracy, precision, recall y medida-f.

**PREGUNTA: De la definición de cada una de las medidas de perfomance (accuracy, precision, recall y medida-f)**

**RESPUESTA:**

Implemente una función 'imprimir_performance' que dado un vector de entrada 'X', un vector de salida 'y', y un clasificador 'clf':
- Realice la predicción para el vector de entrada X.
- Imprima la medida de accuracy.
- Imprima precision, recall y medida f de cada clase.
- Imprima la matriz de confusión.

Para esto puede utilizar el paquete *metrics* de *sklearn*.

In [ ]:
import sklearn.metrics as metrics
import numpy as np

def imprimir_performance(X, y, clf):
    # predicciones = np.array([clf.predict(np.array(x).reshape(1,-1)) for x in X.values])
    predicciones = clf.predict(np.array(X))
        
    print("Accuracy: " + str(metrics.accuracy_score(y, predicciones)))
    
    for l in [' <=50K', ' >50K']:
        print("Label " + l)
        print("   Precision: " + str(metrics.precision_score(y, predicciones, pos_label=l)))
        print("   Recall: " + str(metrics.recall_score(y, predicciones, pos_label=l)))
        print("   Medida-f: " + str(metrics.f1_score(y, predicciones, pos_label=l)))
        
    print("Confussion matrix:\n" + str(metrics.confusion_matrix(y, predicciones)))

Utilice la función **imprimir_performance** para imprimir las medidas de performance para el clasificador **dt** basado en árboles de decisión:

In [ ]:
imprimir_performance(X_test, y_test, dt)

Utilice la función **imprimir_performance** para imprimir las medidas de performance para el clasificador **nb** basado en Naive Bayes:

In [ ]:
imprimir_performance(X_test, y_test, nb)

Utilice la función **imprimir_performance** para imprimir las medidas de performance para el clasificador **svc** basado en Support Vector Machines:

In [ ]:
imprimir_performance(X_test, y_test, svc)

**PREGUNTA: Realice un breve análisis de los resultados obtenidos.**

**RESPUESTA:**

## Validación cruzada

Entrene y mida la perfomance de los calsifificadores anteriores, pero ahora utilizando el algoritmo de validación cruzada (cross validation) tomando 5 particiones. Imprima el promedio de accuracy obtenido para cada modelo:

In [ ]:
import sklearn.model_selection as ms

cross = ms.KFold(n_splits=5)

clfs = [("DT", tree.DecisionTreeClassifier()), ("NB", naive_bayes.GaussianNB()), ("SVC",  svm.SVC())]

performances = {}

for nombre, clf in clfs:
    
    sumAcc = 0
    
    for train, test in cross.split(X_train, y_train):
           
        clf.fit(X_train[train],y_train[train])
        predicciones = clf.predict(X_train[test])
        sumAcc += metrics.accuracy_score(y_train[test], predicciones)
    
    performances[nombre] = sumAcc/5
    
    print("Accuracy " + nombre + ": " + str(sumAcc/5))
    

**PREGUNTA: Describa brevemente cuáles son las ventajas de utilizar validación cruzada en vez de realizar una único esquema de partición como se hizo al principio.**

**RESPUESTA:**

## Mejorando los resultados

Existen varias técnicas que pueden ser utilizadas para mejorar los resultados de nuestros modelos. A continuación utilizaremos técnias de **selección de atributos** y de **ajuste de hiperparámetros**.

## Selección de atributos

En nuestros entrenamientos hemos utilizado todos los atributos disponibles para entrenar nuestros modelos. Pero no siempre esto lleva a los mejores resultados, de hecho muchas veces, trabajar con un conjunto reducido de atributos devuelve mejores resultados.

**PREGUNTA: Investigue de qué se trata la técnica de selección de atributos (feature selection) y argumente brevemente por qué puede mejorar la performance de un algoritmo de aprendizaje automático.**

**RESPUESTA:**

Utilizando el paquete *feature_selection* de *sklearn*, seleccione e imprima la lista de los 20 mejores atributos según la medida estadística chi^2:

In [ ]:
import sklearn.feature_selection as fs

selector = fs.SelectKBest(fs.chi2,k=20)
selector.fit_transform(X_train, y_train).astype(np.int32)

Intente obtener la lista de los mejores N atributos, donde N sea la cantidad mínima posible de atributos que mantenga o mejore las medidas de performance obtenidas con validación cruzada:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def realizarCV(clf, X, y):
    
    cross = ms.KFold(n_splits=5)
    sumAcc = 0
    
    for train, test in cross.split(X, y):
        
        clf.fit(X[train],y[train])
        predicciones = clf.predict(np.array(X[test]))
        sumAcc += metrics.accuracy_score(y[test], predicciones)
    
    return sumAcc / 5

plt.clf()
selectores = {}

for nombre, clf in clfs:

    performanceTarget = realizarCV(clf, X_train, y_train)
    nuevasPerformances = []
    mejoresAtributos = []
    
    for n in xrange(1,20):
        kSelector = fs.SelectKBest(fs.chi2,k=n)
        X_filtered = kSelector.fit_transform(X_train, y_train).astype(np.int32)
        res = realizarCV(clf, X_filtered, y_train)
        
        if len(mejoresAtributos) == 0 and res > performances[nombre]:
            mejoresAtributos = kSelector.get_support()
            selectores[nombre] = kSelector

        nuevasPerformances.append(res)
    
    print(nombre)
    print(mejoresAtributos)
    plt.plot(nuevasPerformances, label=nombre)
    
    
plt.legend()
plt.show()

Con el conjunto de atributos obtenido, entrene los clasificadores nuevamente y verifique que las medidas de precision, recall mejoran en general:

In [ ]:
for nombre, clf in clfs:
    X_selected = selectores[nombre].transform(X_train)
    X_test_selected = selectores[nombre].transform(X_test)
    clf.fit(X_selected,y_train)
    imprimir_performance(X_test_selected, y_test,clf)

## Ajuste de hiperparámetros

Por lo general, cada algoritmo y modelo de aprendizaje automático posee parámetros configurables. Estos parámetros se los suele denominar 'hiperparámetros' del algoritmo, ya que son parámetros que el algoritmo no ajusta automáticamente, sino que son ajustados por el "usuario".

La correcta selección de estos hiperparámetros por lo general tiene una gran incidencia en la performance de los algoritmos.

**PREGUNTA: Para los modelos generados anteriormente (Árbol de decisión, Naive Bayes y Support Vector Machines), investigue en la documentación de scikit-learn cuáles son sus hiperparámetros y qué valores toman. A continuación liste y de una breve descripción de cada uno:**

**RESPUESTA:**


Pruebe diferentes configuraciones de hiperparámetros para los modelos anteriores de modo de mejorar los resultados de performance obtenidos mediante la función *imprimir_performance*.

Para esto puede realizarlo manualmente o buscar una estrategia más avanzada utilizando la clase *GridSearchCV* del paquete *grid_search* de *sklearn*. Esta clase permite definir una grilla de parámetros y posibles valores para luego entrenar el modelo con todas sus posibles combinaciones y devolver la configuración que retorna la mejor performance.

En caso de tener que combinar varios procesos de extracción y selección de atributos junto con un modelo de aprendizaje, se recomienda utilizar la clase *Pipeline* del paquete *pipeline* de *sklearn*.

Tener en cuenta que si la grilla es muy grande, el proceso puede requerir mucho tiempo de cómputo y memoria.

In [ ]:
import sklearn.grid_search as gs

params = {}
params["SVC"] = { 
    'C': [0.5,1,1.5],
    'kernel': ['linear', 'poly', 'rbf'],
}

params["DT"] = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'max_features' : ['auto', 'log2', None],
    'min_samples_split' : [1,2,3,0.1,0.2],
    'min_samples_leaf' : [1,2,3,0.2,0.4],
    'min_impurity_split' : [1e-7,1e-8],
    'min_weight_fraction_leaf' : [0,0.1,0.2]
}
params["NB"] = {
    
}

def obtenerScorer(fn):
    return lambda estimator, X, y : fn(y, estimator.predict(np.array(X)))

def obtenerMejorEstimador(params, scorer):

    for nombre, clf in clfs:

        print("==> " + nombre)
        gridSearch = gs.GridSearchCV(clf, params[nombre], scoring=scorer)
        gridSearch.fit(X_train, y_train)
        pad = " " * len("==>  ")
        print(pad + str(gridSearch.best_score_))
        print(pad + "Best params: " + str(gridSearch.best_params_))

print("Accuracy")
obtenerMejorEstimador(params, obtenerScorer(metrics.accuracy_score))
print("Precision")
obtenerMejorEstimador(params, obtenerScorer(lambda y, p: metrics.precision_score(y, p, pos_label=' <=50K')))
print("Recall")
obtenerMejorEstimador(params, obtenerScorer(lambda y, p: metrics.recall_score(y, p, pos_label=' <=50K')))

**PREGUNTAS:**
- **Cuáles son los valores de los hiperparámetros con los cuales se obtienen los mejores resultados de performance?**
- **Con qué modelo se obtienen los mejores resultados de precision y recall?**

**RESPUESTA:**

**PREGUNTA: Escriba las conclusiones generales que haya obtenido de la tarea.**

**RESPUESTA:**

# Clasificación de Imágenes

En esta sección trabajaremos con clasificación de imágenes. Cada instancia a clasificar es una imagen con un dígito escrito a mano. El objetivo es detectar el dígito correspondiente a cada imagen. Para eso utilizaremos un dataset de *sklearn.datasets* que contiene imágenes de dígitos escritos a mano etiquetadas. Cada imagen se representa como un vector de pixeles.

Utilizar la función *load_digits* para importar los datos de dígitos escritos a mano. Inspeccionar su contenido (data, target, images y target_names), renderizar el dígito de la primera instancia del dataset:

In [ ]:
import sklearn.datasets as datasets

digits = datasets.load_digits()
plt.gray()
plt.matshow(digits.images[0])
plt.show()

Particionar los datos en dos conjuntos dijuntos de entrenamiento y testeo:

In [ ]:
xDigitsTrain,xDigitsTest,yDigitsTrain,yDigitsTest = sklearn.cross_validation.train_test_split(digits.data,digits.target,
                                                                                              test_size =0.5)
len(digits.images)

Extraer atributos de las imágenes para ser utilizados en el modelo de clasificación. Para esto, investigar las clases de Principal Component Analysis (PCA) del paquete sklearn.decomposition:

In [ ]:
import sklearn.decomposition

pca = sklearn.decomposition.PCA(n_components=16)
xDigitsTrain = pca.fit_transform(xDigitsTrain,yDigitsTrain)
xDigitsTest = pca.transform(xDigitsTest)


**PREGUNTA: Explique el método de extracción de atributos y justifique su elección.**

**RESPUESTA:**

Elija dos algoritmos de aprendizaje y entrene e intente obtener los mejores modelos de clasificación posibles:

In [ ]:
from sklearn import svm as svm
from sklearn import neural_network


svc = svm.SVC(gamma=0.001)
svc.fit(xDigitsTrain, yDigitsTrain)
predictSvc = svc.predict(xDigitsTest)

nn = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(20,20), max_iter=80000,alpha=0.001)
nn.fit(xDigitsTrain,yDigitsTrain)




Imprima los mejores resultados de precision, recall y accuracy para los algoritmos seleccionados:

In [ ]:
def imprimir_performance(X, y, clf):
    # predicciones = np.array([clf.predict(np.array(x).reshape(1,-1)) for x in X.values])
    predicciones = clf.predict(X)
        
    print("Accuracy: " + str(metrics.accuracy_score(y, predicciones)))
    

    for l in range(10):
        print("Label " + str(l))
        print("   Precision: " + str(metrics.precision_score(y, predicciones, average = 'micro', labels=[l])))
        print("   Recall: " + str(metrics.recall_score(y, predicciones, average = 'micro', labels=[l])))
        print("   Medida-f: " + str(metrics.f1_score(y, predicciones, average = 'micro', labels=[l])))
        
    print("Confussion matrix:\n" + str(metrics.confusion_matrix(y, predicciones, )))
    
    
imprimir_performance(xDigitsTest,yDigitsTest,svc)
imprimir_performance(xDigitsTest,yDigitsTest,nn)

**PREGUNTA: Analice los resultados obtenidos.**

**RESPUESTA:**